# Residual Network


従来のCNNアーキテクチャでは22層（GoogLeNet）が限界だったが、このアーキテクチャは152層のネットワークを実現した<br>
「ある層で求める***最適な出力***を学習するのではなく、層の入力を参照した***残差関数***を学習する」 というアイデア


ResNetには**残差ブロック**と**Shortcut Connection**が導入されている<br>
一般的なNNが下の図左側とすると、残差ブロックは右側で表すことができる

<img src="NN.png" width="200px"><img src="ResNet.png" width="330px"><br>
残差ブロックでは、畳込み層とSkip Connectionの組み合わせになっている<br>2つの枝から構成されていて、それぞれの要素を足し合わせる<br>残差ブロックの一つはConvolution層の組み合わせで、もう一つはIdentity関数となる<br>
残差ブロックの導入により、より深いネットワークを実現できた


Residual Blockには２つのアーキテクチャがある
 - Plainアーキテクチャ(左)
 - Bottleneckアーキテクチャ(右)

<img src="Plain.png" width="330px"><img src="Bottleneck.png" width="330px"><br>


バッチノーマライゼーションと畳み込み層を繰り返した正規ルートの出力にそれらをスキップしたショートカットを加算した結果をrelu関数に与える
***

**[step 1]**ライブラリのインポート

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms,datasets
import torch.utils.data
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

**[step 2-1]** Plainブロック

In [10]:
class ResidualPlainBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(ResidualPlainBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2d = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if in_channels!=out_channels:
            self.shortcut.add_module('conv', nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0, bias=False))
            self.shortcut.add_module('bn', nn.BatchNorm2d(out_channels))
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)), inplace=True)
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out, inplace=True)

**[step 2-2]** Bottleneckブロック

In [11]:
class ResidualBottleneckBlock(nn.Module):
    expansion=4
    
    def __init__(self, in_channels, out_channels, stride):
        super(ResidualBottleneckBlock, self).__init__()
        
        bottleneck_channels = out_channels//self.expansion
        
        self.conv1 = nn.Conv2d(in_channels, bottleneck_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(bottleneck_channels)
        
        self.conv2 = nn.Conv2d(bottleneck_channels, bottleneck_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(bottleneck_channels)
        
        self.conv3 = nn.Conv2d(bottleneck_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut.add_module('conv', nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0, bias=False))
            self.shortcut.add_module('bn', nn.BatchNorm2d(out_channels))
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)), inplace=True)
        out = F.relu(self.bn2(self.conv2(out)), inplace=True)
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        return F.relu(out, inplace=True)